In [1]:
import sqlite3
import ssl
import urllib.request
import json
import re

hero_re = re.compile(r'HERO_0(\d)')

url = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"
def download_hsjson(url):
    context = ssl._create_unverified_context()
    req = urllib.request.urlopen(url, context=context)
    f = req.read()
    with open('cards.collectible.json', 'wb') as file:
        file.write(f)
    return f

In [2]:
raw_data = download_hsjson(url)

In [3]:
with open('cards.collectible.json', 'r', encoding='utf-8') as f:
    cards_json = json.load(f)

In [4]:
for i in range(10):
    c = cards_json[i]
    print(c['name'])

Commanding Shout
Museum Curator
Ice Rager
Unearthed Raptor
Fireguard Destroyer
Flametongue Totem
Windfury Harpy
Blade Flurry
Force of Nature
Healing Touch


In [5]:
path_to_db = 'stats.db'

In [6]:
tbl_query = r"SELECT * FROM sqlite_master WHERE type='table'"

create_cards_table_sql = r"""CREATE TABLE "cards" (
	`id`	TEXT NOT NULL,
	`name`	TEXT NOT NULL,
	`rarity`	TEXT NOT NULL,
	`cost`	INTEGER NOT NULL,
	`attack`	INTEGER NOT NULL,
	`health`	INTEGER NOT NULL,
	`set`	TEXT,
	PRIMARY KEY(id)
)"""

create_hero_table_sql = r"""CREATE TABLE "hero" (
	`id`	INTEGER NOT NULL UNIQUE,
	`name`	TEXT NOT NULL,
	`cardid`	TEXT NOT NULL,
	`class`	INTEGER NOT NULL,
	PRIMARY KEY(id)
)"""

create_deck_table_sql = r"""CREATE TABLE "deck" (
	`id`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`name`	TEXT NOT NULL,
	`class`	INTEGER NOT NULL,
	`tag1`	INTEGER,
	`tag2`	INTEGER
)"""

create_match_table_sql = r"""CREATE TABLE "match" (
	`id`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`opponent`	INTEGER NOT NULL,
	`first`	INTEGER NOT NULL,
	`won`	INTEGER NOT NULL,
	`duration`	INTEGER NOT NULL,
	`date`	DATETIME NOT NULL,
	`opp_hero`	INTEGER NOT NULL,
	`player_hero`	INTEGER NOT NULL,
	`deck`	INTEGER NOT NULL,
	FOREIGN KEY(`opponent`) REFERENCES `player`(`id`),
	FOREIGN KEY(`opp_hero`) REFERENCES `hero`(`id`),
	FOREIGN KEY(`player_hero`) REFERENCES `hero`(`id`),
	FOREIGN KEY(`deck`) REFERENCES deck(id)
)"""

create_player_table_sql = r"""
CREATE TABLE `player` (
	`id`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`name`	TEXT NOT NULL,
	`high`	INTEGER NOT NULL,
	`low`	INTEGER NOT NULL
)"""

create_cards_played_sql = r"""CREATE TABLE "card_played" (
	`id`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`cardid`	INTEGER NOT NULL,
	`turn`	INTEGER NOT NULL,
	FOREIGN KEY(`cardid`) REFERENCES `cards`(`id`)
)"""

insert_card_sql = r"INSERT INTO `cards`(`id`,`name`,`rarity`,`cost`,`attack`,`health`,`set`) VALUES (?,?,?,?,?,?,?);"

In [7]:
db = sqlite3.connect(path_to_db)
db.row_factory = sqlite3.Row
cursor = db.cursor()
result = cursor.execute(tbl_query)
tables = result.fetchall()
db.close()

In [20]:
db = sqlite3.connect(path_to_db)
db.row_factory = sqlite3.Row
cursor = db.cursor()

In [13]:
with open('db_sql.sql', 'r') as f:
    cursor.executescript(f.read())

In [14]:
db.commit()

In [21]:
def check_table_structure(tables, **kw):
    for table in tables:
        name = table['name']
        print(name)

In [19]:
db.close()

In [22]:
db.row_factory = sqlite3.Row
result = cursor.execute(tbl_query)
tables = result.fetchall()
check_table_structure(tables)

player
sqlite_sequence
match
hero
deck
cards
card_played


In [23]:
def update_card_table(cursor, cards):
    for card in cards:
        result = cursor.execute(r"SELECT id from cards WHERE name = ?", (card['name'],))
        row = result.fetchone()
        if row is None:
            print(card['id'])
            m = hero_re.match(card['id'])
            if m:
                sql_str = 'SELECT * from hero where cardid LIKE ?'
                row = cursor.execute(sql_str, ("%" + card['id'],)).fetchone()
                if row is None:
                    sql_str2 = 'INSERT INTO hero (name, cardid, class) VALUES (?,?,?)'
                    print('Inserting' + card['name'] )
                    cursor.execute(sql_str2, (card['name'], card['id'], m.group(1)))
            ins = (card['id'], card['name'], card['rarity'], card.get('cost', -1),
                  card.get('attack', -1), card.get('health', -1), card['set'])
            cursor.execute(insert_card_sql, ins)


In [24]:
db = sqlite3.connect(path_to_db)
db.row_factory = sqlite3.Row
cursor = db.cursor()
update_card_table(cursor, cards_json)
db.commit()
db.close()

NEW1_036
LOE_006
AT_092
LOE_019
BRM_012
EX1_565
EX1_033
CS2_233
EX1_571
CS2_007
NEW1_014
EX1_279
HERO_04a
InsertingLady Liadrin
AT_121
AT_122
LOE_020
GVG_060
BRM_011
GVG_008
EX1_007
NEW1_041
EX1_001
AT_132
OG_101
OG_335
GVG_014
NEW1_020
FP1_009
CS2_031
CS2_039
OG_280
CS2_092
OG_218
NEW1_018
GVG_107
EX1_028
CS2_088
OG_100
OG_308
OG_323
AT_114
GVG_082
AT_050
AT_075
GVG_050
OG_102
OG_234
EX1_543
CS2_179
PRO_001
AT_062
LOE_115
EX1_029
CS2_181
CS2_213
EX1_005
AT_060
AT_078
CS2_038
OG_113
CS2_013
GVG_073
EX1_354
CS2_118
CS2_104
GVG_019
NEW1_004
LOE_010
EX1_626
AT_055
LOE_105
CS2_042
EX1_616
AT_130
EX1_178
EX1_560
CS2_059
OG_241
EX1_080
LOE_009
AT_016
AT_037
OG_302
AT_020
OG_337
GVG_113
OG_045
AT_044
EX1_573
EX1_562
CS2_075
EX1_124
CS2_063
CS2_227
NEW1_022
AT_064
GVG_091
CS2_146
GVG_054
EX1_563
EX1_025
EX1_355
FP1_001
EX1_593
GVG_100
BRM_031
GVG_057
GVG_010
GVG_058
BRM_015
AT_086
EX1_055
OG_249
EX1_110
EX1_010
OG_330
OG_083
GVG_029
EX1_158
AT_012
GVG_033
LOE_027
OG_042
AT_032
EX1_173
GVG_068


In [ ]:
for card in cards_json:
    print(card['name'])